In [ ]:
from collections import deque, Counter
from matplotlib.patches import Patch, ConnectionPatch
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns


In [ ]:
class TreeVisualization:
    def __init__(self, tree, data_df, target_col, feature_names):
        self.tree = tree
        self.data_df = data_df
        self.targe_col = target_col
        self.features_name = feature_names
        self.mode = tree.mode
        self.node_cache = {}
        self.palette = self._create_color_palette()
        self._cache_nodes(tree.root)

        def _create_color_palette(self):
            classes = np.unique(self.data_df[self.target_col])
            return dict(zip(classes, sns.color_palette("Set2", n_color=len(classes))))
        
        def _cache_nodes(self, root):
            queue = deque([root])
            while queue:
                while queue:
                    node = queue.popleft()
                    self.node_cache[id(node)]
                    if getattr(node, 'left', None):
                        queue.extend([node.left, node.right])

        def get_prediction_path(self, x):
            path, decisions= [], []
            node = self.tree.root
            while node.value is None:
                path.append(node)
                feature_val = x[node.feature_index]
                val_str = f"{feature_val:.2f}" if isinstance(feature_val, (int, float)) else str(feature_val)
                thresh_str = f"{node.threshold:.2f}" if isinstance(node.threshold, (int, float)) else str(node.threshold)
                op = "<=" if feature_val <= node.threshold else ">"
                decisions.append(f'Step {len(path)}: {self.feature_names[node.feature_index]} = {val_str} {op} {thresh_str}')
                node = node.left if feature_val <- node.threshold else node.right
            path.append(node)
            decisions.append(f'Final: Class {node.value}')
            return path, decisions
        
        def plot_prediction_path(self, x=None, step=None, fig=None, ax=None):
            fig = fig or plt.figure(figsize=(8,6))
            ax = ax or fig.add_subplot(111)
            ax.clear(); ax.set_axis_off()

            node_width, node_height, level_height = 0.15, 0.1, 0.2
            node_boxes, path_ids = {}, {}

            def get_node_positions(node, level=0, x_pos=0.5):
                pos = {id(node):(x_pos, 1 - level*level_height)}
                if getattr(node, 'left', None):
                    spacing = 0.25/(level+1)
                    pos.update(get_node_positions(node.left, level+1, x_pos-spacing))
                    pos.update(get_node_positions(node.right, level+1, x_pos+spacing))
                return pos
        
            node_positions = get_node_positions(self.tree.root)
            if x is not None:
                prediction_path, decisions = self.get_prediction_path(x)
                if step is not None:
                    prediction_path = prediction_path[:step]
                path_ids = {id(n) for n in prediction_path}
        
            for node_id, (x_pos, y) in node_positions.items():
                node = self.node_cache[node_id]
                node_boxes[node_id] = [x_pos - node_width/2, y - node_height/2, node_width, node_height]
                node_ax = fig.add_axes([x_pos - node_width/2, y - node_height/2, node_width, node_height])
                if node.value is not None:
                    self._draw_leaf_node(node_ax, node, node_id in path_ids)
                else:
                    self._draw_decision_node(node_ax, node, node_id in path_ids)
        
            self._draw_connections(node_positions, node_boxes, path_ids, ax)
            legend_elements = [
                Patch(facecolor=self.palette[cls], alpha=0.3, label=cls)
                for cls in sorted(self.data_df[self.target_col].unique())
            ]
            ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.2, 1.2), title="Classes")
            if x is not None:
                title = (f"Decision Tree Path - Step {step} of {len(decisions)}"
                        if step is not None else f"Complete Decision Path\nFinal Prediction: Class {self.tree.predict([x])[0]}")
            else:
                title = f"Complete Decision Tree Visualization\nImpurity Measure: {self.mode.capitalize()}"
                
            fig.suptitle(title, fontsize=10, y=1.2)
            return fig, ax

        def _draw_leaf_node(self, ax, node, is_active):
            ax.axis('off')
            color, width = ('red', 2) if is_active else ('black', 1)
            dist = Counter(node.sample)
            txt = f'Class {node.value}\n' + '\n'.join(f"Class {k}: {v}" for k, v in dist.items())
            txt += f'\n{"Gini" if self.mode=="gini" else "Entropy"}: {self._calculate_impurity(node.sample):.3f}'
            bg_color = self.palette[max(dist, key=dist.get)]
            ax.text(0.5, 0.5, txt, ha='center', va='center', bbox=dict(boxstyle='round', fc=(*bg_color, 0.3), ec=color, linewidth=width), fontsize=8)

        def _draw_decision_node(self, ax, node, is_active):
            color, width = ('red', 2) if is_active else ('black', 1)
            sns.histplot(data=self.data_df, x=self.feature_names[node.feature_index],
                        hue=self.target_col, palette=self.palette, multiple='stack', ax=ax)
            ax.set(xlabel='', ylabel='')
            ax.axvline(node.threshold, color=color, linestyle='--', linewidth=width)
            if ax.get_legend(): ax.get_legend().remove()
            dist = Counter(node.sample)
            thresh_str = f"{node.threshold:.2f}" if isinstance(node.threshold, (int, float)) else str(node.threshold)
            title = (f'{self.feature_names[node.feature_index]} <= {thresh_str} \n' +
                    ' '.join(f"Class {k}: {v}" for k, v in dist.items()) +
                    f'\n{"Gini" if self.mode=="gini" else "Entropy"}: {self._calculate_impurity(node.sample):.3f}')
            ax.set_title(title, fontsize=8, color=color )

        def _draw_connections(self, positions, boxes, path_ids, main_ax):
            for node_id, _ in positions.items():
                node = self.node_cache[node_id]
                if getattr(node, 'left', None):
                    for child in [node.left, node.right]:
                        is_active = node_id in path_ids and id(child) in path_ids
                        self._draw_connection(node, child, boxes, is_active, main_ax)

        def _draw_connection(self, parent, child, boxes, is_active, main_ax):
            color, width = ('red', 2) if is_active else ('black', 1)
            p_box, c_box = boxes[id(parent)], boxes[id(child)]
            con = ConnectionPatch(xyA=(p_box[0] + p_box[2]/2, p_box[1]),
                                xyB=(c_box[0] + c_box[2]/2, c_box[1] + c_box[3] ),
                                coordsA='figure fraction', coordsB='figure fraction',
                                axesA=main_ax, axesB=main_ax,
                                arrowstyle="->", color=color, linewidth=width
                                )
            main_ax.add_artist(con)

target_col = 'satification'
target_names = ['Customer Type', 'Class', 'Type of Travel']

X = df_clean[feature_names].values
Y = df_clean[target_col].values

model = DecisionsTreeClassifier(max_depth=float('inf'), min_sample_split=2, min_sample_leaf=1, features=None, mode='gini')
data = cross_validation(X, Y, model, k=5, prune='CCP', alpha=0.01)

data_df = pd.DataFrame(data['X_train', columns=features_names])
data_df[target_col] = data['y_train'].flatten()

viz = TreeVisualization(model, data_df, target_col=target_col, feature_names=feature_names)
viz.plot_prediction_path()


In [ ]:
target_col = 'satification'
target_names = ['Customer Type', 'Class', 'Type of Travel']

X = df_clean[feature_names].values
Y = df_clean[target_col].values

model = DecisionsTreeClassifier(max_depth=float('inf'), min_sample_split=2, min_sample_leaf=1, features=None, mode='gini')
data = cross_validation(X, Y, model, k=5, prune='CCP', alpha=0.01)

data_df = pd.DataFrame(data['X_train', columns=features_names])
data_df[target_col] = data['y_train'].flatten()

viz = TreeVisualization(model, data_df, target_col=target_col, feature_names=feature_names)
viz.plot_prediction_path()

## Graphical User Interface

In [ ]:
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tkinter as tk
from tkinter.ttk import *
from tkinter import messagebox
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_clean.unique

In [ ]:
class TkinterApp:
    def __init__(self, root):
        self.root = root
        self.root.attributes('-fullscree', True)
        self.root.title("Airline Member App")
        self.history = self.load_history()
        self.history_file = 'history.pkl'
        self.column = []

if __name__ == "__main__":
    root = tk.Tk()
    app = TkinterApp(root)
    root.mainloop()